In [19]:
import community as community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd 
import numpy as np
import os.path
import seaborn as sns

pd.options.mode.chained_assignment = None  
sns.set(context='talk', style='white', rc={'figure.facecolor':'white'}, font_scale=1)

# read ibd

read the summed pairwise IBD calls after related individuals have been removed 

In [2]:
ibd = pd.read_csv("ibd.csv", header=None)

In [3]:
ibd.head() # the columns here are 0: the pair, 1: the total amount of ibd, 2: the total amount of segments shared, 3: ID1, 4: ID2, 5: the pair again for some reason

,0,1,2,3,4,5
0,"('0_HGDP00837', '0_S_Surui-2')",3024.25587,187,0_HGDP00837,0_S_Surui-2,"('0_HGDP00837', '0_S_Surui-2')"
1,"('0_HGDP00832', '0_HGDP00837')",2850.37362,196,0_HGDP00832,0_HGDP00837,"('0_HGDP00832', '0_HGDP00837')"
2,"('0_HGDP00837', '0_HGDP00849')",2646.35156,176,0_HGDP00837,0_HGDP00849,"('0_HGDP00837', '0_HGDP00849')"
3,"('0_HGDP00837', '0_HGDP00843')",2606.71754,144,0_HGDP00837,0_HGDP00843,"('0_HGDP00837', '0_HGDP00843')"
4,"('0_HGDP00832', '0_HGDP00843')",2563.07138,169,0_HGDP00832,0_HGDP00843,"('0_HGDP00832', '0_HGDP00843')"


In [4]:
# take only the ID1, ID2, and the total amount of IBD shared between the pair
# make a matrix for clustering 
# if you only have these three columns you can start here 

ibd_matrix = ibd[[3, 4, 1]].values 
ibd = [] # removes ibd df from memory since it is no longer needed 

# make graph
make undirected graph where each individual is a node of the graph and each edge is weighted by the total amount of shared IBD 

In [2]:
edges_list = list(map(tuple, ibd_matrix)) # getting list of nodes and edges in format for networkx 

In [6]:
G = nx.Graph() # defining an empty graph with networkx 

In [7]:
G.add_weighted_edges_from(edges_list) # add edge weights and nodes to the graph 

In [ ]:
edges_list = [] # removes from memory 

# initial louvain partitioning

run louvain clustering over the graph for the first iteration 

In [ ]:
partition = community_louvain.best_partition(G) # run louvain clustering and return a partition (cluster assignment for each node)

In [ ]:
max(partition.values()) # the number of clusterss

In [ ]:
partition_df = pd.DataFrame.from_dict(partition,  orient="index").reset_index() # make the partition into a dataframe

In [19]:
partition_df.to_csv("louvain.csv", index=False, header=False) # save dataframe

# subpartition 
create subclusters from the initial louvain partitioning 
(this step will be relatively time consuming depending on the number of clusters you have) 

In [4]:
# iterate over every partition in the initial clustering
for cluster in partition_df[0].unique(): 
    
    nodes = partition_df[partition_df[0] == cluster]["index"].values # extract the nodes belonging to a given cluster
    H = G.subgraph(nodes) # create a new graph only with the nodes from that cluster 
    
    subpartition = community_louvain.best_partition(H) # run louvain again for that subgraph 
    
    subpartition_df = pd.DataFrame.from_dict(subpartition,  orient="index").reset_index() # create dataframe and save
    subpartition_df.to_csv(f"louvain_subcluster{cluster}.csv", index=False)

    # rerun for an additional level of clustering 
    for subcluster in subpartition[0].unique(): 
        nodes = subpartition_df[subpartition_df[0] == subcluster]["index"].values
        I = H.subgraph(nodes) 

        final_partition = community_louvain.best_partition(H)
        
        final_partition_df = pd.DataFrame.from_dict(final_partition,  orient="index").reset_index() 
        final_partition_df.to_csv(f"louvain_subcluster{cluster}_{subcluster}.csv", index=False)
    

# cluster properties 
you may also be interested in what properties the clusters have in network space 

In [135]:
cluster_num = 1 # pick a cluster of interest 
nodes = partition_df[partition_df[0] == cluster_num][0].values # pick a partition of interest 

In [136]:
H = G.subgraph(nodes) # make the subgraph of the cluster you are interested in 

## degree centrality 
the number of connections a node has (i.e., the number of people in the cluster an individual shares IBD with) 

In [3]:
# plot histogram 
plt.hist(list(nx.degree_centrality(H).values()), bins=20, color="#FF9AA2")
plt.xlabel("degree centrality")
plt.ylabel("number of nodes")
plt.title(f"cluster{cluster_num}")
plt.show()